# Age & Gender Detection for Horror Roller Coaster Entry
**Objective**: Detect age and gender in real-time. If age < 13 or > 60, display “Not allowed” with a red box and log all entries to a CSV file.

In [ ]:
# Import libraries
import cv2
import numpy as np
import datetime
import pandas as pd
from tensorflow.keras.models import load_model

In [ ]:
# Load pre-trained models
# You should train your own models or use lightweight versions for real-time inference
# For demonstration, this uses placeholder models
# Replace with your own model paths
age_model = load_model("age_model.h5")  # Your age detection model
gender_model = load_model("gender_model.h5")  # Your gender detection model

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
# Prepare logging
log_file = "rollercoaster_log.csv"
df = pd.DataFrame(columns=["Age", "Gender", "Entry Time"])
df.to_csv(log_file, index=False)

In [ ]:
# Start webcam and detect faces
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face_img, (64, 64)) / 255.0
        face_input = np.expand_dims(face_resized, axis=0)

        # Predict age and gender
        age = int(age_model.predict(face_input)[0][0])
        gender_pred = gender_model.predict(face_input)
        gender = "Male" if gender_pred[0][0] > 0.5 else "Female"

        color = (0, 255, 0)
        label = f"Age: {age}, Gender: {gender}"

        if age < 13 or age > 60:
            label += " - Not allowed"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Log to CSV
        time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_data = pd.DataFrame([[age, gender, time_now]], columns=["Age", "Gender", "Entry Time"])
        new_data.to_csv(log_file, mode='a', header=False, index=False)

    cv2.imshow("Age & Gender Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()